In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [35]:
regularseasonraw = pd.read_csv('MDataFiles_Stage1/MRegularSeasonDetailedResults.csv')
postseasonraw = pd.read_csv('MDataFiles_Stage1/MNCAATourneyDetailedResults.csv')
rankings = pd.read_csv('MDataFiles_Stage1/MMasseyOrdinals.csv')

In [4]:
def add_rate_cols(df):
    
    df['W_possessions'] = .5 * (df['WFGA'] + (.475 * df['WFTA']) - df['WOR'] + df['WTO'])
    df['WeFG'] = (df['WFGM'] + (.5 * df['WFGM3'])) / df['WFGA']
    df['WTSpct'] = df['WScore'] / (2 * (df['WFGA'] + .475 * df['WFTA']))
    df['WFTR'] = df['WFTA'] / df['WFGA']
    df['W3PR'] = df['WFGA3'] / (df['WFGA'] + (.475*df['WFTA']))
    df['WORpct'] = df['WOR'] / (df['WOR'] + df['LDR'])
    df['WDRpct'] = df['WDR'] / (df['WDR'] + df['LOR'])
    df['WREBpct'] = (df['WOR'] + df['WDR']) / (df['WOR'] + df['LDR'] + df['LOR'] + df['WDR'])
    df['WTR'] = (df['WOR'] + df['WDR'])
    df['WATOr'] = df['WAst'] / df['WTO']
    df['WAst_pct'] = df['WAst'] / df['WFGM']
    df['WStl_pct'] = df['WStl'] / (df['LFGA'] + .475*df['LFTA'] + df['LTO'])
    df['WBlk_pct'] = df['WBlk'] / (df['LFGA'])
    df['WTO_ratio'] = df['WTO'] / (df['WFGA'] + .475*df['WFTA'] + df['WTO'])
    df['L_possessions'] = .5 * (df['LFGA'] + (.475 * df['LFTA']) - df['LOR'] + df['LTO'])
    df['LeFG'] = (df['LFGM'] + (.5 * df['LFGM3'])) / df['LFGA']
    df['LTSpct'] = df['LScore'] / (2 * (df['LFGA'] + .475 * df['LFTA']))
    df['LFTR'] = df['LFTA'] / df['LFGA']
    df['L3PR'] = df['LFGA3'] / (df['LFGA'] + (.475*df['LFTA']))
    df['LORpct'] = df['LOR'] / (df['LOR'] + df['WDR'])
    df['LDRpct'] = df['LDR'] / (df['LDR'] + df['WOR'])
    df['LREBpct'] = (df['LOR'] + df['LDR']) / (df['WOR'] + df['LDR'] + df['LOR'] + df['WDR'])
    df['LTR'] = (df['LOR'] + df['LDR'])
    df['LATOr'] = df['LAst'] / df['LTO']
    df['LAst_pct'] = df['LAst'] / df['LFGM']
    df['LStl_pct'] = df['LStl'] / (df['WFGA'] + .475*df['WFTA'] + df['WTO'])
    df['LBlk_pct'] = df['LBlk'] / (df['WFGA'])
    df['LTO_ratio'] = df['LTO'] / (df['LFGA'] + .475*df['LFTA'] + df['LTO'])
    
    """df = df[['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
       'W_possessions', 'WeFG', 'WTSpct', 'WFTR', 'W3PR', 'WORpct', 'WDRpct',
       'WREBpct', 'WTR', 'WATOr', 'WAst_pct', 'WStl_pct', 'WBlk_pct',
       'WTO_ratio', 'L_possessions', 'LeFG', 'LTSpct', 'LFTR', 'L3PR',
       'LORpct', 'LDRpct', 'LREBpct', 'LTR', 'LATOr', 'LAst_pct', 'LStl_pct',
       'LBlk_pct', 'LTO_ratio']]
    """
    
    return df

In [5]:
regularseason = regularseasonraw
postseason = postseasonraw

In [6]:
regularseason.DayNum.value_counts().sort_index()

0        2
1      186
2       41
3       20
4      381
      ... 
128    496
129    914
130    688
131    369
132     83
Name: DayNum, Length: 133, dtype: int64

In [36]:
regularseasonraw.WLoc.value_counts(normalize = True)

H    0.592824
A    0.306284
N    0.100892
Name: WLoc, dtype: float64

In [8]:
df = regularseason
df.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

In [9]:
def team_regular_season(team_id,Season = all, DayNum = all, average = True):
    
    teamwins = regularseason[(regularseason['WTeamID'] == team_id) & (regularseason['Season'] == Season) & (regularseason['DayNum'] < DayNum)] 
    teamlosses = regularseason[(regularseason['LTeamID'] == team_id) & (regularseason['Season'] == Season) & (regularseason['DayNum'] < DayNum)]
    
    teamwinsoffense = teamwins.filter(like = 'W')    
    teamlossesoffense = teamlosses.filter(like = 'L')

    teamwinsoffense.columns = teamwinsoffense.columns.str.lstrip('W')
    teamlossesoffense.columns = teamlossesoffense.columns.str.lstrip('L')

    teamoffense = pd.concat([teamwinsoffense, teamlossesoffense])

    teamwinsopponents = teamwins.filter(like = 'L')
    teamlossesopponents = teamlosses.filter(like = 'W')

    teamwinsopponents.columns = teamwinsopponents.columns.str.lstrip('L')
    teamlossesopponents.columns = teamlossesopponents.columns.str.lstrip('W')

    teamopponents = pd.concat([teamwinsopponents, teamlossesopponents])
    
    teamtotal = pd.concat([teamoffense, teamopponents.add_prefix('Opp_')], axis = 1)
    
    ###cols_to_drop = ['Def_TeamID', 'Def_Score',]
    
    if average == True:
        teamtotal = teamtotal.groupby('TeamID').mean().reset_index()
        
    return teamtotal


In [10]:
marquette = team_regular_season(1266, 2020, 132, True)
marquette

,TeamID,Score,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,...,Opp_FGA3,Opp_FTM,Opp_FTA,Opp_OR,Opp_DR,Opp_Ast,Opp_TO,Opp_Stl,Opp_Blk,Opp_PF
0,1266,77.766667,25.3,58.866667,10.033333,26.233333,17.133333,23.066667,10.7,29.266667,...,21.366667,14.2,21.366667,10.2,24.833333,13.166667,10.966667,6.766667,3.066667,19.466667


In [11]:
marquette.columns

Index(['TeamID', 'Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR',
       'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'Opp_TeamID', 'Opp_Score',
       'Opp_FGM', 'Opp_FGA', 'Opp_FGM3', 'Opp_FGA3', 'Opp_FTM', 'Opp_FTA',
       'Opp_OR', 'Opp_DR', 'Opp_Ast', 'Opp_TO', 'Opp_Stl', 'Opp_Blk',
       'Opp_PF'],
      dtype='object')

In [12]:
def regular_season_game(team1_id, team2_id, Year = 2020, DayNum = 132):
    team1 = team_regular_season(team1_id,Year,DayNum)
    team2 = team_regular_season(team2_id,Year,DayNum)
    game = pd.concat([team1, team2], axis = 1)
    
    return game

duke_unc = regular_season_game(1181, 1314)

In [13]:
duke_unc.columns

Index(['TeamID', 'Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR',
       'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'Opp_TeamID', 'Opp_Score',
       'Opp_FGM', 'Opp_FGA', 'Opp_FGM3', 'Opp_FGA3', 'Opp_FTM', 'Opp_FTA',
       'Opp_OR', 'Opp_DR', 'Opp_Ast', 'Opp_TO', 'Opp_Stl', 'Opp_Blk', 'Opp_PF',
       'TeamID', 'Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR',
       'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'Opp_TeamID', 'Opp_Score',
       'Opp_FGM', 'Opp_FGA', 'Opp_FGM3', 'Opp_FGA3', 'Opp_FTM', 'Opp_FTA',
       'Opp_OR', 'Opp_DR', 'Opp_Ast', 'Opp_TO', 'Opp_Stl', 'Opp_Blk',
       'Opp_PF'],
      dtype='object')

In [45]:
rankings.columns

Index(['Season', 'RankingDayNum', 'SystemName', 'TeamID', 'OrdinalRank'], dtype='object')

In [82]:
KenPom = rankings[(rankings['SystemName'] == 'POM') & (rankings['Season'] == 2020) & (rankings['RankingDayNum'] == 128)]
KenPom = KenPom[['TeamID', 'OrdinalRank']]

In [84]:
KenPom.shape

(353, 2)

In [85]:
regularseasonraw.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

In [99]:
schedule2020 = regularseasonraw[(regularseasonraw['Season'] == 2020) & (regularseasonraw['DayNum'] > 12)].loc[:,['WTeamID', 'LTeamID', 'DayNum','Season']]
print(schedule2020.shape)
schedule2020 = schedule2020.merge(KenPom, how = 'inner', left_on = 'WTeamID', right_on = 'TeamID')
schedule2020 = schedule2020.merge(KenPom, how = 'inner', left_on = 'LTeamID', right_on = 'TeamID')
schedule2020['Fav_Win'] = schedule2020['OrdinalRank_x'] < schedule2020['OrdinalRank_y']
schedule2020 = schedule2020.sort_values('DayNum')
schedule2020

(4795, 4)


,WTeamID,LTeamID,DayNum,Season,TeamID_x,OrdinalRank_x,TeamID_y,OrdinalRank_y,Fav_Win
0,1112,1308,13,2020,1112,20,1308,91,True
281,1114,1146,13,2020,1114,130,1146,280,True
2897,1394,1410,13,2020,1394,297,1410,247,False
2776,1392,1295,13,2020,1392,184,1295,129,False
371,1332,1426,13,2020,1332,17,1426,131,True
...,...,...,...,...,...,...,...,...,...
1012,1393,1314,128,2020,1393,55,1314,84,True
1082,1155,1274,128,2020,1155,73,1274,104,True
108,1112,1449,128,2020,1112,20,1449,53,True
1810,1170,1451,128,2020,1170,189,1451,271,True


In [101]:
schedule = list(zip(schedule2020.WTeamID,schedule2020.LTeamID,schedule2020.DayNum,schedule2020.Season,schedule2020.Fav_Win))

In [102]:
len(schedule2020.WTeamID.unique())

343

In [103]:
len(schedule2020.LTeamID.unique())

344

In [ ]:
### warning 4.5 minutes to run

games2020 = pd.DataFrame()
for i in range(len(schedule)):
    games2020 = games2020.append(regular_season_game(schedule[i][0], schedule[i][1], DayNum = schedule[i][2]))

In [37]:
games2020

,level_0,index,TeamID,Score,FGM,FGA,FGM3,FGA3,FTM,FTA,...,Opp_FTM,Opp_FTA,Opp_OR,Opp_DR,Opp_Ast,Opp_TO,Opp_Stl,Opp_Blk,Opp_PF,team1_win
0,0,0,1112,89.333333,32.333333,61.333333,8.000000,19.000000,16.666667,23.333333,...,14.000000,18.500000,6.500000,21.500000,7.500000,14.000000,7.500000,2.000000,25.500000,1
1,1,0,1113,80.500000,27.000000,65.000000,7.500000,23.500000,19.000000,25.500000,...,13.000000,18.500000,8.500000,25.000000,11.500000,12.500000,8.500000,1.500000,21.000000,1
2,2,0,1114,68.500000,27.000000,52.500000,4.500000,16.500000,10.000000,13.500000,...,11.333333,16.000000,13.000000,37.666667,20.666667,15.000000,9.000000,5.000000,19.000000,1
3,3,0,1117,64.666667,24.000000,51.333333,5.000000,14.666667,11.666667,16.000000,...,9.333333,11.333333,8.000000,23.666667,15.000000,12.666667,5.666667,2.333333,16.666667,1
4,4,0,1123,71.000000,28.500000,66.500000,8.000000,25.000000,6.000000,13.500000,...,20.500000,27.500000,10.500000,27.500000,17.000000,12.500000,4.000000,4.000000,19.500000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4790,4790,0,1389,66.689655,22.724138,55.344828,6.206897,17.827586,15.034483,20.896552,...,12.714286,19.821429,11.571429,23.678571,14.821429,12.928571,6.857143,2.785714,17.357143,1
4791,4791,0,1393,74.096774,25.322581,58.258065,8.129032,24.709677,15.322581,20.580645,...,13.187500,17.343750,8.937500,25.218750,13.750000,11.562500,6.812500,4.218750,18.750000,1
4792,4792,0,1402,70.137931,24.931034,56.862069,5.172414,16.310345,15.103448,20.137931,...,11.133333,15.533333,9.733333,25.833333,13.566667,13.933333,6.933333,2.800000,19.400000,1
4793,4793,0,1412,66.310345,24.034483,53.517241,4.896552,15.827586,13.344828,18.724138,...,16.250000,22.821429,9.714286,29.428571,13.285714,13.392857,5.535714,3.571429,18.357143,1
